In [1]:
import copernicusmarine as cm
import xarray as xr
import polars as pl
import numpy as np
from datetime import datetime
import sys
import os
sys.path.append(os.path.abspath("../python/utils"))
from get_data import make_request_and_open_copernicus

/home/pablo/miniconda3/envs/proy3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
temp = pl.read_csv("../../data/temporal_subset.csv")
box = pl.read_csv("../../data/world_box.csv")

In [3]:
day = temp.filter(pl.col("mas_actual") == 0)["day"].to_list()[0]
month = temp.filter(pl.col("mas_actual") == 0)["month"].to_list()[0]
year = temp.filter(pl.col("mas_actual") == 0)["year"].to_list()[0]
min_time = f"{day}-{month}-{year}"
day = temp.filter(pl.col("mas_actual") == 1)["day"].to_list()[0]
month = temp.filter(pl.col("mas_actual") == 1)["month"].to_list()[0]
year = temp.filter(pl.col("mas_actual") == 1)["year"].to_list()[0]
max_time = f"{day}-{month}-{year}"
fecha_min = datetime.strptime(min_time, "%d-%m-%Y")
fecha_min_format = fecha_min.strftime("%Y-%m-%dT%H:%M:%S")
fecha_max = datetime.strptime(max_time, "%d-%m-%Y")
fecha_max_format = fecha_max.strftime("%Y-%m-%dT%H:%M:%S")
print("min", fecha_min_format)
print("max", fecha_max_format)

min 1999-06-15T00:00:00
max 2015-10-07T00:00:00


## Fecha mínima provisional

Para trabajar con un archivo más manejable, usaremos un subset más pequeño y luego al final usaremos el completo.

In [4]:
fecha_min_provisional = "9-10-2015"
fecha_min_provisional = datetime.strptime(fecha_min_provisional, "%d-%m-%Y")
fecha_min_format = fecha_min_provisional.strftime("%Y-%m-%dT%H:%M:%S")

In [5]:
max_latitude = box["max_latitude"].to_list()[0]
min_latitude = box["min_latitude"].to_list()[0]
max_longitude = box["max_longitude"].to_list()[0]
min_longitude = box["min_longitude"].to_list()[0]
print("max", max_latitude, max_longitude)
print("min", min_latitude, min_longitude)

max -61.767468 179.988159
min -78.173 -179.998241


## Request

In [6]:
data = make_request_and_open_copernicus("cmems_mod_glo_phy_my_0.083deg_P1M-m", "Global_Ocean_Physics_Reanalysis.nc",
                                        fecha_min_format, fecha_max_format, max_latitude, min_latitude, max_longitude, min_longitude)
data

INFO - 2025-03-10T10:58:01Z - Downloading Copernicus Marine data requires a Copernicus Marine username and password, sign up for free at: https://data.marine.copernicus.eu/register


Copernicus Marine username:Copernicus Marine password:

INFO - 2025-03-10T10:58:11Z - Selected dataset version: "202311"
INFO - 2025-03-10T10:58:11Z - Selected dataset part: "default"
WARNING - 2025-03-10T10:58:14Z - Some of your subset selection [0, 1] for the depth dimension exceed the dataset coordinates [0.49402499198913574, 5727.9169921875]
INFO - 2025-03-10T10:58:17Z - Starting download. Please wait...
100%|██████████| 66/66 [00:04<00:00, 15.73it/s]
INFO - 2025-03-10T10:58:21Z - Successfully downloaded to /home/pablo/Desktop/zird/2/proy/github/data/copernicus/Global_Ocean_Physics_Reanalysis_(1).nc


<xarray.Dataset> Size: 75MB
Dimensions:    (depth: 1, latitude: 197, longitude: 4319, time: 1)
Coordinates:
  * depth      (depth) float32 4B 0.494
  * latitude   (latitude) float32 788B -78.17 -78.08 -78.0 ... -61.92 -61.83
  * longitude  (longitude) float32 17kB -179.9 -179.8 -179.8 ... 179.8 179.9
  * time       (time) datetime64[ns] 8B 2015-10-01
Data variables:
    bottomT    (time, latitude, longitude) float64 7MB ...
    mlotst     (time, latitude, longitude) float64 7MB ...
    siconc     (time, latitude, longitude) float64 7MB ...
    sithick    (time, latitude, longitude) float64 7MB ...
    so         (time, depth, latitude, longitude) float64 7MB ...
    thetao     (time, depth, latitude, longitude) float64 7MB ...
    uo         (time, depth, latitude, longitude) float64 7MB ...
    usi        (time, latitude, longitude) float64 7MB ...
    vo         (time, depth, latitude, longitude) float64 7MB ...
    vsi        (time, latitude, longitude) float64 7MB ...
    zos        (time, latitude, longitude) float64 7MB ...
Attributes:
    references:                http://marine.copernicus.eu
    contact:                   servicedesk.cmems@mercator-ocean.eu
    credit:                    E.U. Copernicus Marine Service Information (CM...
    institution:               Mercator Ocean
    title:                     Monthly mean fields for product GLOBAL_REANALY...
    producer:                  CMEMS - Global Monitoring and Forecasting Centre
    source:                    MERCATOR GLORYS12V1
    Conventions:               CF-1.6
    copernicusmarine_version:  2.0.1

In [7]:
data.to_dataframe()

bottomT     mlotst    siconc  \
depth    latitude   longitude   time                                        
0.494025 -78.166664 -179.916672 2015-10-01       NaN        NaN       NaN   
                    -179.833328 2015-10-01       NaN        NaN       NaN   
                    -179.750000 2015-10-01       NaN        NaN       NaN   
                    -179.666672 2015-10-01       NaN        NaN       NaN   
                    -179.583328 2015-10-01       NaN        NaN       NaN   
...                                              ...        ...       ...   
         -61.833332  179.583328 2015-10-01  0.546495  64.088873  0.007515   
                     179.666672 2015-10-01  0.545030  56.611838  0.007630   
                     179.750000 2015-10-01  0.544298  54.628135  0.009880   
                     179.833328 2015-10-01  0.543565  52.644431  0.008011   
                     179.916672 2015-10-01  0.543565  50.660728  0.007096   

                                             sithick         so    thetao  \
depth    latitude   longitude   time                                        
0.494025 -78.166664 -179.916672 2015-10-01       NaN        NaN       NaN   
                    -179.833328 2015-10-01       NaN        NaN       NaN   
                    -179.750000 2015-10-01       NaN        NaN       NaN   
                    -179.666672 2015-10-01       NaN        NaN       NaN   
                    -179.583328 2015-10-01       NaN        NaN       NaN   
...                                              ...        ...       ...   
         -61.833332  179.583328 2015-10-01  0.039674  33.751946 -0.223304   
                     179.666672 2015-10-01  0.048830  33.745842 -0.311197   
                     179.750000 2015-10-01  0.060274  33.742790 -0.403485   
                     179.833328 2015-10-01  0.049593  33.748894 -0.473800   
                     179.916672 2015-10-01  0.048830  33.756523 -0.533860   

                                                  uo       usi        vo  \
depth    latitude   longitude   time                                       
0.494025 -78.166664 -179.916672 2015-10-01       NaN       NaN       NaN   
                    -179.833328 2015-10-01       NaN       NaN       NaN   
                    -179.750000 2015-10-01       NaN       NaN       NaN   
                    -179.666672 2015-10-01       NaN       NaN       NaN   
                    -179.583328 2015-10-01       NaN       NaN       NaN   
...                                              ...       ...       ...   
         -61.833332  179.583328 2015-10-01  0.072634 -0.000031  0.224006   
                     179.666672 2015-10-01  0.087283  0.000092  0.241707   
                     179.750000 2015-10-01  0.100101  0.000214  0.249641   
                     179.833328 2015-10-01  0.111087  0.000153  0.244758   
                     179.916672 2015-10-01  0.119633  0.000061  0.225837   

                                                 vsi       zos  
depth    latitude   longitude   time                            
0.494025 -78.166664 -179.916672 2015-10-01       NaN       NaN  
                    -179.833328 2015-10-01       NaN       NaN  
                    -179.750000 2015-10-01       NaN       NaN  
                    -179.666672 2015-10-01       NaN       NaN  
                    -179.583328 2015-10-01       NaN       NaN  
...                                              ...       ...  
         -61.833332  179.583328 2015-10-01  0.000183 -1.255226  
                     179.666672 2015-10-01  0.001221 -1.267129  
                     179.750000 2015-10-01  0.001099 -1.279641  
                     179.833328 2015-10-01  0.000702 -1.292459  
                     179.916672 2015-10-01  0.000000 -1.304361  

[850843 rows x 11 columns]